In [1]:
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine, text
from pandas.tseries.offsets import BDay

engine = create_engine('mysql+pymysql://root:@localhost:3306/portfolio_development')
conpf = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()
engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

In [3]:
today = date.today()
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'today: {today}')
print(f'yesterday: {yesterday}')

today: 2025-09-03
yesterday: 2025-09-02


In [5]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\Downloads\Datasets
dts_path = os.path.join(user_path, "Downloads", "Datasets")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")
#C:\Users\PC1\OneDrive\A5\Excel
xsl_path = os.path.join(base_path, "Excel")

In [7]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path (dat_path): {dat_path}") 
print(f"Excel path (xsl_path): {xsl_path}") 
print(f"Google Drive path (god_path): {god_path}")
print(f"iCloudDrive path (icd_path): {icd_path}") 
print(f"Obsidian path (osd_path): {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path (dat_path): C:\Users\PC1\OneDrive\A5\Data
Excel path (xsl_path): C:\Users\PC1\OneDrive\A5\Excel
Google Drive path (god_path): C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path (icd_path): C:\Users\PC1\iCloudDrive\Data
Obsidian path (osd_path): C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [9]:
format_dict = {
    'shares':'{:,}',    
    'price':'{:.2f}',
    'dividend':'{:.4f}', 
    'date':'{:%Y-%m-%d}', 
    
    'qty':'{:,}','shares':'{:,}',
    'price':'{:.2f}','buy_price':'{:.2f}',
    'dividend':'{:.4f}',    
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}',
  
    'days':'{:,}',
    'price':'{:.2f}',
    'fee':'{:,.2f}','vat':'{:,.2f}','net':'{:,.2f}','profit':'{:,.2f}',
    'percent':'{:,.2f}%','yearly':'{:,.2f}%',   
    
    'shares':'{:,}',    
    'q4':'{:.4f}','q3':'{:.4f}','q2':'{:.4f}','q1':'{:.4f}','dividend':'{:.4f}',
    'xdate':'{:%Y-%m-%d}','paiddate':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','max_price':'{:.2f}','min_price':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    }

In [11]:
def show_buy(const, name):
    sql = """
    SELECT * 
    FROM buy 
    WHERE name = '{}'
    """.format(name)
    
    df_buy = pd.read_sql(sql, const)
    df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
    df_buy.rename(columns={'volbuy':'shares'},inplace=True)
    df_buy['shares'] = df_buy['shares'].astype('int64')
    return df_buy.style.format(format_dict).hide(axis="index")

In [13]:
def show_dividend(const, name):
    sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
    """.format(name)
    
    df_dividend = pd.read_sql(sql, const)
    df_dividend.drop(['PRICE', 'PERCENT'], axis=1, inplace=True)
    df_dividend.columns = df_dividend.columns.str.lower()
    df_dividend['shares'] = df_dividend['shares'].astype('int64')
    df_dividend['xdate'] = pd.to_datetime(df_dividend['xdate'])
    df_dividend['paiddate'] = pd.to_datetime(df_dividend['paiddate'])
    return df_dividend.style.format(format_dict).hide(axis="index")

In [15]:
def show_stocks(conlite, name):
    sql = """
    SELECT * 
    FROM stocks 
    WHERE name = '{}'
    """.format(name)
    
    df_stocks = pd.read_sql(sql, conlite)
    return df_stocks.style.format(format_dict).hide(axis="index")

In [17]:
def update_buy(const, name, new_qty, new_price):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE buy
        SET volbuy = :new_qty,
        price = :new_price
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,
        'new_price': new_price,
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [19]:
def update_dividend(const, name, new_qty):
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE dividend
        SET shares = :new_qty
        WHERE name = :name
    """)
    
    # Execute the query with parameters
    result = const.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'name': name
    })

    return f"Records updated = {result.rowcount}"

In [21]:
def update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty):  
    # Use parameterized query to avoid SQL injection
    sqlUpd = text("""
        UPDATE stocks
        SET available_qty = :new_qty, 
            cost = :new_unit_cost,
            buy_target = :new_buy_target,
            sell_target = :new_sell_target,    
            qty = :new_buy_qty
        WHERE name = :name
    """)    
    # Execute the query with parameters
    result = conlite.execute(sqlUpd, {
        'new_qty': new_qty,  # Use the scalar value
        'new_unit_cost': new_unit_cost, # Use the scalar value 
        'new_buy_target': new_buy_target,
        'new_sell_target': new_sell_target,     
        'new_buy_qty': new_buy_qty,
        'name': name
    })
    return f"Records updated = {result.rowcount}"

### End of Definition

## Begin of Sale transaction

In [25]:
# Sells table in MySQL portfolio database
sql = """
SELECT name, stock_id, B.date AS buy_date, qty, B.price AS buy_price, S.* 
FROM sells S
JOIN buys B ON S.buy_id = B.id
JOIN stocks T ON B.stock_id = T.id
ORDER BY S.id DESC
LIMIT 1"""
df_sells_latest = pd.read_sql(sql, conpf)
df_sells_latest.style.format(format_dict).hide(axis="index")

name,stock_id,buy_date,qty,buy_price,id,buy_id,date,price,fee,vat,net,days,profit,percent,yearly,sequence,chart,dividend_id
DIF,81,2025-04-09,"5,000",7.50,1112,1319,2025-09-03,8.30,85.91,6.01,"41,408.08",147,"3,825.02",10.18%,25.27%,-112,None,0


### In case of sale, retreive buy id from sells record (buy_id)

In [28]:
# Buys table in MySQL portfolio database
transaction = 'S'

names = df_sells_latest['name']
name = names.to_string(index=False)

sr_qty = df_sells_latest['qty']
qty = sr_qty.to_string(index=False)
qty = int(qty) * -1

sr_price = df_sells_latest['buy_price']
buy_price = sr_price.to_string(index=False)
buy_price = float(buy_price)

sr_price = df_sells_latest['price']
sell_price = sr_price.to_string(index=False)
sell_price = float(sell_price)

buy_ids = df_sells_latest['buy_id']
buy_id = buy_ids.to_string(index=False)
print(name, qty, buy_price, sell_price)

DIF -5000 7.5 8.3


### Begin of Update buy table in MySQL stock database from sale transaction

In [31]:
sql = """
    SELECT *
    FROM buy 
    WHERE name = '{}'
""".format(name)
    
df_buy = pd.read_sql(sql, const)
df_buy.drop(['volsell', 'volbal','dividend'], axis=1, inplace=True)
df_buy.rename(columns={'volbuy':'shares'},inplace=True)
df_buy['shares'] = df_buy['shares'].astype('int64')
df_buy.style.format(format_dict).hide(axis="index")

name,date,shares,price,active,period,grade
DIF,2020-08-01,"45,000",12.69,1,2,C1


In [30]:
old_qty = int(df_buy['shares'].iloc[0])
old_price = df_buy['price'].iloc[0]
old_qty, old_price 

(50000, 12.17)

In [33]:
sql = """
    SELECT name, B.* 
    FROM buys B
    JOIN stocks T 
    ON B.stock_id = T.id
    WHERE name = '{}' AND status = 'Active'
    ORDER BY id DESC
""".format(name)
df_buys = pd.read_sql(sql, conpf)
# Format all floats (pandas 1.4.0+)
df_buys.style.format(format_dict).hide(axis="index")

name,id,stock_id,date,qty,price,fee,vat,status,net,kind,chart
DIF,1338,81,2025-08-14,"10,000",8.45,174.92,12.24,Active,"84,687.16",HD,None
DIF,1312,81,2024-11-08,"5,000",9.10,94.19,6.59,Active,"45,600.78",HD,None
DIF,824,81,2021-03-02,"20,000",14.70,608.58,42.60,Active,"294,651.18",HD,None
DIF,813,81,2021-02-17,"10,000",14.70,304.29,21.30,Active,"147,325.59",HD,None


In [35]:
# Calculate total price*qty and total qty
total_price_qty = (df_buys['price'] * df_buys['qty']).sum()
total_qty = df_buys['qty'].sum()

# Compute average unit cost
average_unit_cost = round(total_price_qty / total_qty,2)

print(f"Total qty: {total_qty:,.0f}")
print(f"Average Unit Cost: {average_unit_cost:.2f}")

Total qty: 45,000
Average Unit Cost: 12.69


In [37]:
old_cost_amt = df_buy['shares'].iloc[0] * df_buy['price'].iloc[0]
sales_amt = qty * sell_price
new_qty = total_qty
new_cost_amt = total_price_qty
new_unit_cost = average_unit_cost
print(name, old_cost_amt, sales_amt, new_cost_amt, new_qty, new_unit_cost)

DIF 571000.5 -41500.0 571000.0 45000 12.69


In [39]:
show_buy(const, name)

name,date,shares,price,active,period,grade
DIF,2020-08-01,"45,000",12.69,1,2,C1


In [41]:
print(name, new_qty, new_unit_cost)

DIF 45000 12.69


In [53]:
new_unit_cost = 12.70

In [55]:
update_buy(const, name, new_qty, new_unit_cost)

'Records updated = 1'

In [57]:
show_buy(const, name)

name,date,shares,price,active,period,grade
DIF,2020-08-01,"45,000",12.70,1,2,C1


In [47]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM buy
    WHERE name = :name AND volbuy = 0 AND active = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


### End of Update buy table in MySQL stock database from sale transaction

### Begin of Update dividend table in MySQL stock database from sale transaction

In [51]:
sql = """
    SELECT * 
    FROM dividend 
    WHERE name = '{}'
""".format(name)
    
df_dividend = pd.read_sql(sql, const)
df_dividend.columns = df_dividend.columns.str.lower()
df_dividend['shares'] = df_dividend['shares'].astype('int64')

df_dividend.style.format(format_dict).hide(axis="index")

name,q4,q3,q2,q1,dividend,price,percent,shares,xdate,paiddate,kind,actual
DIF,0.2222,0.2222,0.2222,0.2222,0.8888,0.00,0.00%,"45,000",2025-08-15,2025-09-09,,1


In [52]:
new_qty = df_dividend['shares'].iloc[0] + qty
new_qty

45000

In [54]:
show_dividend(const, name)

name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
DIF,0.2222,0.2222,0.2222,0.2222,0.8888,"50,000",2025-08-15,2025-09-09,,1


In [90]:
update_dividend(const, name, new_qty)

'Records updated = 1'

In [92]:
show_dividend(const, name)

name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
DIF,0.2222,0.2222,0.2222,0.2222,0.8888,"45,000",2025-08-15,2025-09-09,,1


In [94]:
# Define the SQL query
sqlDel = text("""
    DELETE FROM dividend
    WHERE name = :name AND shares = 0 AND actual = 0;
""")

# Execute the query with the correct parameter dictionary
result = const.execute(sqlDel, {'name': name})

# Print the number of rows deleted
print(f"Records deleted: {result.rowcount}")  

Records deleted: 0


In [96]:
show_dividend(const, name)

name,q4,q3,q2,q1,dividend,shares,xdate,paiddate,kind,actual
DIF,0.2222,0.2222,0.2222,0.2222,0.8888,"45,000",2025-08-15,2025-09-09,,1


### End of Update dividend table in MySQL stock database from sale transaction

### Begin of Update stocks table in SQLite port_lite database from sale transaction

In [72]:
sql = """
    SELECT name, cost, available_qty
    FROM stocks
    WHERE name = '{}'
""".format(name)

df_stocks = pd.read_sql(sql, conlite)
df_stocks['available_qty'] = df_stocks['available_qty'].astype('int64')
df_stocks

,name,cost,available_qty
0,DIF,12.17,50000


In [74]:
old_qty = int(df_stocks['available_qty'].iloc[0])
new_qty = old_qty + qty
new_buy_target = 7.80
new_sell_target = 12.70
new_buy_qty = 5000

In [76]:
show_stocks(conlite, name)

id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
107,DIF,0.00,0.00,S,8.30,8.30,0.00,0.00,12.17,"5,000",-4,4,"50,000",0.000000,0.000000,RD40Pct,SET


In [78]:
print(name, transaction, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

DIF S 45000 12.688888888888888 7.8 12.7 5000


In [80]:
update_stock(conlite, name, new_qty, new_unit_cost, new_buy_target, new_sell_target, new_buy_qty)

'Records updated = 1'

In [82]:
show_stocks(conlite, name)

id,name,max_price,min_price,status,buy_target,sell_target,volume,beta,cost,qty,buy_spread,sell_spread,available_qty,bl,sh,reason,market
107,DIF,0.00,0.00,S,7.80,12.70,0.00,0.00,12.69,"5,000",-4,4,"45,000",0.000000,0.000000,RD40Pct,SET


### End of Update stocks table in SQLite port_lite database from sale transaction

## End of Sale process

### =====================================================

In [122]:
#price_date = '2025-02-14'
sql = """
    SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, \
    FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,\
    FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt, \
    FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol, \
    FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol \
    FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' \
    ORDER BY period, buy.name
""".format(yesterday)
print(sql)


    SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date,     FORMAT(volbuy,0) AS volbuy, FORMAT(buy.price,2) AS buy_price, price.price AS mkt_price,    FORMAT((volbuy * buy.price),2) AS amtbuy, FORMAT((volbuy * price.price),2) AS amtmkt,     FORMAT(((price.price - buy.price) * volbuy),2) AS amtpol,     FORMAT((((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100),2) AS pctpol     FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '2025-09-02'     ORDER BY period, buy.name



In [124]:
sql = """
    SELECT period, buy.grade AS grade, buy.name AS name, buy.date AS date, 
    volbuy AS volbuy, buy.price AS buy_price, price.price AS mkt_price,
    (volbuy * buy.price) AS amtbuy, (volbuy * price.price) AS amtmkt, 
    ((price.price - buy.price) * volbuy) AS amtpol, 
    (((price.price - buy.price)*volbuy)/(volbuy*buy.price)*100) AS pctpol 
    FROM buy INNER JOIN price ON buy.name = price.name WHERE price.date = '{}' 
    ORDER BY period, buy.name
""".format(yesterday)

df_buy = pd.read_sql(sql, const)

# Now apply formatting in pandas with correct numeric types
format_dict = {
    'volbuy': '{:,.0f}',
    'buy_price': '{:,.2f}',
    'mkt_price': '{:,.2f}',
    'amtbuy': '{:,.2f}',
    'amtmkt': '{:,.2f}',
    'amtpol': '{:,.2f}',
    'pctpol': '{:,.2f}%'
}

df_buy.style.format(format_dict).hide(axis="index")

period,grade,name,date,volbuy,buy_price,mkt_price,amtbuy,amtmkt,amtpol,pctpol
1,C1,AH,2023-06-08,"1,200",37.00,14.20,"44,400.00","17,040.00","-27,360.00",-61.62%
1,A1,AWC,2023-06-15,"9,000",4.96,2.26,"44,640.00","20,340.00","-24,300.00",-54.44%
1,A2,JMART,2023-01-20,"6,800",33.00,8.50,"224,400.00","57,800.00","-166,600.00",-74.24%
1,C2,ORI,2022-04-07,"50,000",9.00,2.22,"450,000.00","111,000.00","-339,000.00",-75.33%
1,C2,PTTGC,2021-03-17,"6,000",64.75,27.75,"388,500.00","166,500.00","-222,000.00",-57.14%
1,C2,SCC,2021-09-20,600,405.00,216.00,"243,000.00","129,600.00","-113,400.00",-46.67%
1,A3,SINGER,2023-01-19,"6,000",24.80,5.70,"148,800.00","34,200.00","-114,600.00",-77.02%
2,A1,AIMIRT,2023-08-17,"17,500",10.60,9.75,"185,500.00","170,625.00","-14,875.00",-8.02%
2,C1,CPNREIT,2022-08-16,"55,000",18.00,11.30,"990,000.00","621,500.00","-368,500.00",-37.22%
2,C1,DIF,2020-08-01,"45,000",12.69,8.25,"571,000.50","371,250.00","-199,750.50",-34.98%


In [126]:
file_name = 'portfolio.csv'
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [128]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\portfolio.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\portfolio.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\portfolio.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\portfolio.csv


In [130]:
df_buy.to_csv(output_file, header=True, index=False)
df_buy.to_csv(icd_file, header=True, index=False)
df_buy.to_csv(god_file, header=True, index=False)
df_buy.to_csv(osd_file, header=True, index=False)

In [132]:
file_name = '035-portfolio.xlsx'
xsl_file = os.path.join(xsl_path, file_name)
df_buy.to_excel(xsl_file, index=False)